# Obtain the block_ranking

In [1]:
import pandas as pd
import pickle
import networkx as nx
import queue
import itertools
import numpy as np
import time

Load the graph by the pickle file created through the RQ1:

In [2]:
with open("Graph", "rb") as f:
    graph = pickle.load(f)

In [3]:
df = pd.read_csv('wiki-topcats-page-names.txt',sep='\n', header=-1, names=["Name"])

#remove the index from the name:
df["Name"]=df.apply(lambda x: " ".join(x["Name"].split()[1:]),axis=1)
df.head()

,Name
0,Chiasmal syndrome
1,Kleroterion
2,Pinakion
3,LyndonHochschildSerre spectral sequence
4,Zariski's main theorem


In [4]:
n=len(df)
n_graph = graph.number_of_nodes()
print("Total number of nodes:",n)
print("Number of nodes by the graph:",n_graph)

Total number of nodes: 1791489
Number of nodes by the graph: 461193


We oserve that the great majority of the nodes are not in the graph (isolated points)

The structure of the file containing the categories is:

    Category:NameOfTheCategory; indexes of the nodes belonging to the category separeted by space

In [5]:
category = pd.read_csv("wiki-topcats-categories.txt", header = None, sep = "/n", names=["all"],engine="python")

#Reaching the name of the category:
#Splitting by ";" and the taking the first part, splitting by ":" and taking the second part
category["Name"]=category.apply(lambda x: x["all"].split(";")[0].split(":")[1], axis=1)

#Reaching the list pf nodes:
#Splitting by ":" and taking the second part and split by " "
category["nodes"]= category.apply(lambda x: list(map(int,x["all"].split(";")[1].split())),axis=1)

#Just change the order of the columns and drop the columns "all"
category= category[["Name","nodes"]]

category.head()

,Name,nodes
0,Buprestoidea,"[301, 302, 303, 304, 305, 306, 307, 308, 309, ..."
1,People_from_Worcester,"[1056, 1057, 1058, 1059, 1060, 60971, 76515, 7..."
2,Skin_conditions_resulting_from_physical_factors,"[971, 973, 1166, 1167, 1168, 1169, 1170, 1171,..."
3,Visual_kei_bands,"[1297, 1300, 1311, 1312, 1313, 1314, 1315, 131..."
4,Japanese_rock_music_groups,"[1297, 1300, 1313, 1314, 1315, 1316, 1319, 132..."


Now we can keep (as required) just the category with more than 3500 nodes:

In [6]:
cat = category.loc[category.apply(lambda x: len(x["nodes"]), axis = 1) > 3500]

In [7]:
c = len(cat)
c

35

In [8]:
cat.head()

,Name,nodes
868,English_footballers,"[22860, 28411, 28961, 28979, 29264, 29573, 295..."
869,The_Football_League_players,"[14003, 23536, 27109, 27348, 27459, 27989, 280..."
876,Association_football_forwards,"[26876, 26877, 26879, 26887, 26892, 26904, 269..."
898,Association_football_goalkeepers,"[26900, 26909, 26917, 26960, 26966, 26984, 270..."
900,Association_football_midfielders,"[14003, 15291, 23536, 26880, 26882, 26885, 268..."


In [9]:
lengths=[]
for i,val in cat.iterrows():
    lengths.append(len(val["nodes"]))

In [10]:
lengths

[9237,
 9467,
 6959,
 3997,
 8270,
 6668,
 418223,
 3760,
 6154,
 6580,
 6546,
 5913,
 7851,
 3813,
 34721,
 7729,
 13704,
 5701,
 4853,
 3501,
 4551,
 22699,
 15302,
 3697,
 4888,
 3542,
 11661,
 13938,
 8401,
 12174,
 7237,
 6767,
 3588,
 4040,
 4517]

In order to reduce the running time, let's take the smallest category as the input category:

In [11]:
idx_min =np.argmin(np.array(lengths))
C0 = cat.iloc[idx_min]["Name"]
print(C0)  #Name of the input category

English_television_actors


In [12]:
nodes_C0 = list(cat.loc[cat["Name"]==C0]["nodes"])[0]
print(len(nodes_C0))  #Number of its nodes

3501


Write the function that computes the BFS. That algorithm computes the shortest path between one source node and all the other nodes of the graph for an unweighted graph

In [13]:
def breadth_first_search(G,a):
    visited=[False]*n
    visited[a]=True
    q = queue.Queue()
    q.put(a)
    d={}
    d[a]=0
    while not q.empty():
        v=q.get()
        for i in list(G.neighbors(v)):
            if not visited[i]:
                q.put(i)
                visited[i]=True
                d[i] = d[v]+1
    return d

In [14]:
a = time.time()
bfs1 =nx.single_source_shortest_path_length(G=graph,source=52)
b = time.time()
our_bfs =breadth_first_search(graph, 52)
c = time.time()
print(bfs1 == our_bfs)
print("The running time of the algorithm from the library is",b-a)
print("The running time of our algorithm",c-b)

True
The running time of the algorithm from the library is 1.4900486469268799
The running time of our algorithm 1.559196949005127


Our BFS is identical to the algorithm from the library (so it is correct). Also we see that the running times are very similar.

Now we need to compute the dictionary of the distances (using BFS) with source every node belonging to the category $C_0$. 

We would create a dictionary of this format:

- The key are all the nodes of the graph
- For each key the value is a list, where each element is the lenght of the shortest path with one of the nodes of the category $C_0$

But we would encounter a memory error (the previous dictionary has about 400k keys, each one with value a list of about 3.5k elements).

Then we store as values (instead of a list) a nested dictionary, with keys the distances and values the FREQUENCIES with which each distance appear.

In [16]:
# !!! This cell would take 2 hours, DO NOT RUN IT !!!
# Instead if you want you can upload the "final_dict" (2 cells later)

final_dict=dict((i,{}) for i in graph.nodes)
for i in nodes_C0:
    if i in graph:
        d = breadth_first_search(graph,i)
        for j in d:
            final_dict[j][d[j]] = final_dict[j].get(d[j],0) +1

In [17]:
with open("final_dict","wb") as f:
    pickle.dump(final_dict,f)

Now we can load it:

In [15]:
with open("final_dict","rb") as f:
    final_dict = pickle.load(f)

Let's see how it looks like, just for two nodes:

In [16]:
{95: final_dict[95], 103:final_dict[103], 104:final_dict[104]}

{95: {5: 1835, 6: 1080, 4: 266, 7: 59, 3: 4, 8: 2, 9: 1},
 103: {},
 104: {6: 1771, 5: 1236, 7: 160, 8: 13, 4: 64, 3: 2, 9: 1}}

Now for each category $C_i$, we can easily obtain the __ShortestPath($C_0$,$C_i$)__ just taking all the dictionaries for every node belonging to $C_i$, and merging them.

Once obtained the set  ShortestPath($C_0$,$C_i$), we just need to take the median of it.

We could compute the median using `statistics.median`, but in the code of that function it is used sorted(list) instead of list.sort().

Now the problem is that for the category Living_people which has 400k nodes, the list shortest_path is very large, and if we use sorted() we are creating a copy of this, that could create a __Memory error__. So we have to create a little function to compute the median by ourself:

In [18]:
def median(l):
    l.sort()
    middle = len(l)//2
    return l[middle]

In [19]:
medians={}
list_of_names = cat.loc[cat["Name"]!=C0]["Name"]
for Ci in list_of_names:
    nodes_Ci = list(cat.loc[cat["Name"]==Ci]["nodes"])[0]
    shortest_path=[]
    for node in nodes_Ci:
        try:
            d = final_dict[node]
            for lenght in d:
                shortest_path.extend(d[lenght]*[lenght])
        except:
            continue
    medians[Ci]= median(shortest_path)

In [23]:
medians

{'English_footballers': 6,
 'The_Football_League_players': 6,
 'Association_football_forwards': 6,
 'Association_football_goalkeepers': 6,
 'Association_football_midfielders': 6,
 'Association_football_defenders': 6,
 'Living_people': 6,
 'Year_of_birth_unknown': 6,
 'Harvard_University_alumni': 6,
 'Major_League_Baseball_pitchers': 6,
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 6,
 'Indian_films': 6,
 'Year_of_death_missing': 6,
 'English_cricketers': 6,
 'Year_of_birth_missing_(living_people)': 6,
 'Rivers_of_Romania': 7,
 'Main_Belt_asteroids': 8,
 'Asteroids_named_for_people': 7,
 'English-language_albums': 5,
 'British_films': 5,
 'English-language_films': 5,
 'American_films': 5,
 'Fellows_of_the_Royal_Society': 6,
 'People_from_New_York_City': 5,
 'American_Jews': 5,
 'American_television_actors': 5,
 'American_film_actors': 5,
 'Debut_albums': 6,
 'Black-and-white_films': 5,
 'Year_of_birth_missing': 6,
 'Place_of_birth_missing_(living_people)': 6,
 

In [21]:
with open("medians","wb") as f:
    pickle.dump(medians,f)

In [18]:
with open("medians","rb") as f:
    medians = pickle.load(f)

Just sort the dictionary by values:

In [19]:
block_ranking = [C0]+[k for k, v in sorted(medians.items(), key=lambda x: x[1])]

In [20]:
block_ranking

['English_television_actors',
 'English-language_albums',
 'British_films',
 'English-language_films',
 'American_films',
 'People_from_New_York_City',
 'American_Jews',
 'American_television_actors',
 'American_film_actors',
 'Black-and-white_films',
 'Article_Feedback_Pilot',
 'American_military_personnel_of_World_War_II',
 'English_footballers',
 'The_Football_League_players',
 'Association_football_forwards',
 'Association_football_goalkeepers',
 'Association_football_midfielders',
 'Association_football_defenders',
 'Living_people',
 'Year_of_birth_unknown',
 'Harvard_University_alumni',
 'Major_League_Baseball_pitchers',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'Indian_films',
 'Year_of_death_missing',
 'English_cricketers',
 'Year_of_birth_missing_(living_people)',
 'Fellows_of_the_Royal_Society',
 'Debut_albums',
 'Year_of_birth_missing',
 'Place_of_birth_missing_(living_people)',
 'Windows_games',
 'Rivers_of_Romania',
 'Asteroids_named_for_peop

# Sort the nodes in each category

Before starting take a look that:

In [21]:
bool=False
for i in final_dict:
    if i not in graph:
        bool = True
        break
bool

False

There are no nodes reached by some node (in `final_dict`), but without retiring edges (not in `graph`, since we builded it in this way)

Well, first of all we need to keep every node just to one category, the closest to the input category:

In [22]:
newCat =pd.DataFrame(block_ranking,columns=["Name"])
newCat.head()

,Name
0,English_television_actors
1,English-language_albums
2,British_films
3,English-language_films
4,American_films


In [23]:
already_seen=set() #keep the list of the nodes already seen, starting by C0
nodes2 = []
for i,categ in enumerate(block_ranking): #C0,C1,....
    idx = cat.loc[cat["Name"]==C0].index[0] #take the index
    updated_nodes = []
    for node in list(cat.loc[cat["Name"]==categ]["nodes"])[0]:
        if (node not in already_seen) and (node in graph):  #remove as well nodes not in graph
            already_seen.add(node)
            updated_nodes.append(node)
    nodes2.append(updated_nodes)    

In [24]:
newCat["nodes"]=nodes2
newCat.head()

,Name,nodes
0,English_television_actors,"[32782, 40338, 40566, 53495, 72636, 83294, 840..."
1,English-language_albums,"[185, 186, 40523, 40524, 40525, 40528, 60647, ..."
2,British_films,"[214, 1084, 23768, 28836, 30602, 30714, 35063,..."
3,English-language_films,"[134, 153, 1083, 1087, 1089, 1152, 1158, 1178,..."
4,American_films,"[173, 1131, 13360, 14130, 14138, 15933, 17458,..."


Now we can start

In [25]:
weights = {i:1 for i in graph.nodes}
for ind,value in newCat.iterrows():
    nodes = value[1]
    subgraph = graph.subgraph(nodes)
    newWeights={}
    for node in nodes:
        edges = list(subgraph.in_edges(node))
        newWeights[node] = sum([weights[i[1]] for i in edges])
    #update weights
    for j in newWeights:
        weights[j] = newWeights[j]

We have created the weight for each node, now we just need to sort each category using those weights

In [26]:
Rank=[]
for ind,value in newCat.iterrows():
    nodes = value[1]
    #obtain the dictionary d of weights just for these nodes:
    d={}
    for node in nodes:
        d[node] = weights[node]
    #sort the dictionary d by values and keep just the keys:
    rank_i =[k for k, v in sorted(d.items(), key=lambda x: x[1])]
    Rank.extend(rank_i)

In [27]:
len(Rank)

461193

Just observe that is the same as number of nodes found in RQ1